### Installing BookNLP

In [1]:
!pip install booknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
  Created wheel for booknlp: filename=booknlp-1.0.7-py3-none-any.whl size=2420743 sha256=e22b2deaec4b56f97fd308c17754f9f4874138333c119051a9805b4e71a00b5c
  Stored in directory: /root/.cache/pip/wheels/bb/f5/b4/2ee6b317da6491ff01b742be603af81ceea156427f1f4be38a
Successfully built booknlp


### Extracting the datasets

In [ ]:
!unzip '/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip'

Archive:  /content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip
   creating: OriginalStoriesSeparated/
  inflating: OriginalStoriesSeparated/Babylon5.txt  
  inflating: OriginalStoriesSeparated/DoctorWhoSpinoffs.txt  
  inflating: OriginalStoriesSeparated/DoctorWho.txt  
  inflating: OriginalStoriesSeparated/Fringe.txt  
  inflating: OriginalStoriesSeparated/Farscape.txt  
  inflating: OriginalStoriesSeparated/Futurama.txt  
  inflating: OriginalStoriesSeparated/Stargate.txt  
  inflating: OriginalStoriesSeparated/StarTrek.txt  
  inflating: OriginalStoriesSeparated/StarWarsBooks.txt  
  inflating: OriginalStoriesSeparated/StarWarsRebels.txt  
  inflating: OriginalStoriesSeparated/XFiles.txt  
  inflating: scifi-test.txt          
  inflating: scifi-train.txt         
  inflating: scifi-val.txt           


### BookNLP Implementation + Sentiment Extraction

1) Implementing BookNLP on the datasets \\
2) Extraction of sentences with two people in it connected using a verb \\
3) Getting the sentiment using NLTK Sentiment Analyzer 

In [2]:
# Creating the pipeline - Installing the Models necessary for different processing done by BookNLP
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)

using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


--- startup: 103.748 seconds ---


In [3]:
# Input file to process
input_file="/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/SampleBabylon.txt"

# Output directory to store resulting files in
output_directory="test_booknlp/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="sample_babylon"

# Process the file 
booknlp.process(input_file, output_directory, book_id)

--- spacy: 1.486 seconds ---
--- entities: 29.542 seconds ---
--- quotes: 0.017 seconds ---
--- attribution: 2.125 seconds ---
--- name coref: 0.022 seconds ---
--- coref: 41.735 seconds ---
--- TOTAL (excl. startup): 74.993 seconds ---, 2951 words


In [5]:
import pandas as pd

In [ ]:
# Loading the .tokens file in a table format for extraction of sentences
df_sentences = pd.read_table('/content/test_booknlp/sample_babylon.tokens')
sentences = df_sentences.groupby(['sentence_ID'])['word'].apply(lambda x: ' '.join(x)).values

In [26]:
sentences = sentences.tolist()

In [27]:
len(sentences)

132

In [42]:
lengths = [len(x.split()) for x in sentences]

In [44]:
# Calculating the cumulative lengths of the sentences 
for i in range(len(lengths)):
  if i!=0:
    lengths[i] = lengths[i]+lengths[i-1]

In [10]:
# .entities file consists of different components such as category, start token, etc that will be used for extraction 
df = pd.read_table('/content/test_booknlp/sample_babylon.entities')

In [47]:
text_values = df.text.values
ref_id = df.COREF.values 
cat = df.cat.values
start_token_id = df.start_token.values
type_of_identity = df.prop.values

In [11]:
# .verb file will be utilized to filter sentences where the identities are connected with an verb
df_verb = pd.read_table('/content/test_booknlp/sample_babylon.supersense')

In [28]:
# Importing the NLTK Sentiment Analyzer VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [71]:
# Dictionary to hold identity paris along with the sentences in which they were referenced
paired_sent = {}

In [72]:
# Extraction of sentences with two identities connected using a verb
for j in range(len(sentences)):
  check = False
  identities = []
  for i in range(len(text_values)):
     # Do not want to check for identities that are referenced ebfore or after the current sentence
    if start_token_id[i]<lengths[j-1] and i!=0:
      continue
    if start_token_id[i]>lengths[j]:
      break
    # Identities should only be PER (person) and referred as a PROP (Proper Noun)
    if text_values[i] in sentences[j] and cat[i]=='PER' and type_of_identity[i]=="PROP":
      identities.append([text_values[i], ref_id[i]])
  if len(identities)==2 and identities[1][0]!=identities[0][0]:
    check = True
  for idx, row in df_verb.iterrows():
    if "verb.social" in row.supersense_category and check:
      if row.start_token>lengths[j]:
        break  
      if row.start_token<lengths[j-1] and i!=0:
        continue
      # Collection of all sentences for the pair extracted earlier
      if f'{identities[0][0]} and {identities[1][0]}' in paired_sent or f'{identities[1][0]} and {identities[0][0]}' in paired_sent:
        if paired_sent.get(f'{identities[0][0]} and {identities[1][0]}')!=None:
          paired_sent[f'{identities[0][0]} and {identities[1][0]}'] += " "+sentences[j]
          break
        else:
          paired_sent[f'{identities[1][0]} and {identities[0][0]}'] += " "+sentences[j]
          break
      else:
        paired_sent[f'{identities[0][0]} and {identities[1][0]}'] = ""
        paired_sent[f'{identities[0][0]} and {identities[1][0]}'] += " "+sentences[j]
        break

In [73]:
paired_sent

{'Ambassador Londo Mollari and Kosh': ' Ambassador Londo Mollari quickly becomes a suspect in the case , as he had failed to show up at the appointed time with the other ambassadors to welcome Kosh aboard .',
 'Lyta and Kosh': ' Takashima persuade Lyta to perform an unauthorized mind scan on the unconscious Kosh .',
 'Lyta and Sinclair': ' Lyta accuses Sinclair of attempting to murder the ambassador .',
 "Ambassador G'Kar and Sinclair": " The Advisory Council considers a motion made by Ambassador G'Kar to turn Sinclair over to the Vorlon Empire , to stand trial for attempted murder .",
 'Ambassador Delenn and Ambassador Mollari': ' Takashima votes no ; Ambassador Delenn abstains ; Ambassador Mollari votes in favor .',
 'Sinclair and Mollari': ' Sinclair also tells Mollari he is calling Mollari Council and the League , hopeful he can form a coalition against the Narn attack .',
 'Sinclair and Garibaldi': ' After leaving the garden , Sinclair is met by Garibaldi , who has discovered that

In [78]:
# Getting a sentiment for the pairs, according to the sentences extracted
relations = {'neu':"neutral",'pos':"positive",'neg':"negative"}
inter_char_relations = []
for key in paired_sent:
  ss = SentimentIntensityAnalyzer().polarity_scores(paired_sent[key])
  del ss['compound']
  inter_char_relations.append(f'{key} have {relations[max(ss, key=ss.get)]} relationship')

In [79]:
# Inter character relationships 
inter_char_relations

['Ambassador Londo Mollari and Kosh have neutral relationship',
 'Lyta and Kosh have neutral relationship',
 'Lyta and Sinclair have negative relationship',
 "Ambassador G'Kar and Sinclair have neutral relationship",
 'Ambassador Delenn and Ambassador Mollari have neutral relationship',
 'Sinclair and Mollari have neutral relationship',
 'Sinclair and Garibaldi have neutral relationship',
 "Ambassador Delenn and G'Kar have neutral relationship",
 'Sinclair and the Raiders have neutral relationship',
 'Garibaldi and Mollari have neutral relationship',
 'Ivanova and Winters have neutral relationship']